In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import glob

In [24]:
import requests

rsid = "rs1438898"
url = f"https://grch37.rest.ensembl.org/variation/human/{rsid}?"
headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)
data = response.json()
data

{'evidence': ['Frequency',
  '1000Genomes',
  'Cited',
  'Phenotype_or_Disease',
  'TOPMed',
  'gnomAD'],
 'ambiguity': 'H',
 'source': 'Variants (including SNPs and indels) imported from dbSNP',
 'mappings': [{'start': 145714354,
   'coord_system': 'chromosome',
   'location': '2:145714354-145714354',
   'allele_string': 'A/C/T',
   'end': 145714354,
   'ancestral_allele': 'A',
   'assembly_name': 'GRCh37',
   'seq_region_name': '2',
   'strand': 1}],
 'var_class': 'SNP',
 'MAF': 0.160343,
 'minor_allele': 'C',
 'most_severe_consequence': 'intron_variant',
 'name': 'rs1438898',
 'synonyms': ['rs17479658']}

In [22]:
data

{'most_severe_consequence': 'intron_variant',
 'ambiguity': 'N',
 'minor_allele': 'A',
 'source': 'Variants (including SNPs and indels) imported from dbSNP',
 'mappings': [{'strand': 1,
   'seq_region_name': '1',
   'ancestral_allele': 'G',
   'location': '1:44076469-44076469',
   'coord_system': 'chromosome',
   'assembly_name': 'GRCh37',
   'allele_string': 'G/A/C/T',
   'start': 44076469,
   'end': 44076469}],
 'MAF': 0.474441,
 'synonyms': ['rs60234110'],
 'evidence': ['Frequency',
  '1000Genomes',
  'Cited',
  'Phenotype_or_Disease',
  'TOPMed',
  'gnomAD'],
 'var_class': 'SNP',
 'name': 'rs549845'}

In [9]:
population_groups = {
    "AFR": ["ACB", "ASW", "ESN", "GWD", "LWK", "MSL", "YRI"],
    "AMR": ["CLM", "MXL", "PEL", "PUR"],
    "EAS": ["CDX", "CHB", "CHS", "JPT", "KHV"],
    "EUR": ["CEU", "FIN", "GBR", "IBS", "TSI"],
    "SAS": ["BEB", "GIH", "ITU", "PJL", "STU"]
}
rs_ids = pd.read_csv('../data/snp_coordinates_grch37.tsv', sep='\t')
iHS_scored = pd.DataFrame(columns=['continent', 'country', 'iHS_score', 'chromosome', 'rsID'])

In [12]:
rs_ids.head()

,rsID,Chromosome,Position_GRCh37
0,rs549845,1,44076469
1,rs1438898,2,145714354
2,rs2886697,3,20724204
3,rs9877066,3,43691501
4,rs7613360,3,49916710


In [29]:
pop = "ACB"
chromosome = "2"
#position = "43610798"
rsID = "rs17479658"
path = f"/Users/emilevancauwenberghe/Documents/LargeFiles/Database/ihs_hapbin_globalCM/{pop}/hapbin_globalCM/{pop}.chr{chromosome}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.noCNV.snv.modAncDer.bcfCheck.hapbin.ihs.txt"
ihs_data = pd.read_csv(path, sep='\t')
ihs_data.loc[ihs_data['Location'] == rsID, 'iHS'].values
ihs_data

,Location,iHH_0,iHH_1,iHS,Std iHS
0,rs4499454,0.003582,0.004259,-0.173268,-0.753956
1,rs10200966,0.001939,0.006732,-1.244690,-0.790556
2,rs11680031,0.001953,0.006868,-1.257410,-0.745092
3,rs300761,0.002124,0.008573,-1.395550,-0.901225
4,rs6734688,0.008804,0.003427,0.943706,-0.046338
...,...,...,...,...,...
567866,rs113543697,0.000898,0.002153,-0.874992,1.464250
567867,rs76379979,0.000316,0.001105,-1.252180,-2.570720
567868,rs78145032,0.001091,0.000328,1.200580,3.058250
567869,rs62620233,0.000929,0.000479,0.662528,2.896000


In [ ]:


rows = []
for continent, populations in population_groups.items():
    for pop in populations:
        for _, row in rs_ids.iterrows():
            rsID = row['rsID']
            chromosome = row['Chromosome']
            
            path = f"/Users/emilevancauwenberghe/Documents/LargeFiles/Database/ihs_hapbin_globalCM/{pop}/hapbin_globalCM/{pop}.chr{chromosome}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.noCNV.snv.modAncDer.bcfCheck.hapbin.ihs.txt"
            ihs_data = pd.read_csv(path, sep='\t')
            match = ihs_data.loc[ihs_data['Location'] == rsID, 'iHS'].values
            iHS_score = match[0] if len(match) > 0 else None
            rows.append({
                'continent': continent,
                'country': pop,
                'iHS_score': iHS_score,
                'chromosome': chromosome,
                'rsID': rsID
            })
iHS_scored = pd.DataFrame(rows)